In [461]:
import pandas as pd
import numpy as np
import category_encoders as ce

from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

In [462]:
X=pd.read_csv('train_values.csv',header=0,na_values='?')
y=pd.read_csv('train_labels.csv',header=0,na_values='?')

In [465]:
X.describe()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area,height,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.0000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.00000,10000.000000,10000.000000,10000.000000,10000.0000,10000.000000
mean,9987.160000,7.135600,296.930300,2678.617900,2.146700,25.393500,38.438100,4.653100,0.089700,0.762600,0.02990,0.019000,0.068800,0.072500,0.2561,0.087700,0.040000,0.013800,0.014100,0.984600,0.108600,0.067300,0.029400,0.006400,0.00070,0.00070,0.000800,0.000200,0.000200,0.0001,0.005300
std,5800.800829,6.225567,279.390651,2520.663769,0.736365,64.482893,21.265883,1.792842,0.285766,0.425511,0.17032,0.136532,0.253126,0.259327,0.4365,0.282872,0.195969,0.116666,0.117909,0.423297,0.311152,0.250553,0.168933,0.079748,0.02645,0.02645,0.028274,0.014141,0.014141,0.0100,0.072612
min,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,6.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
25%,4998.750000,2.000000,60.000000,606.750000,2.000000,10.000000,26.000000,4.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
50%,9963.500000,6.000000,219.000000,1937.500000,2.000000,15.000000,34.000000,5.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
75%,15044.750000,10.000000,466.000000,4158.000000,3.000000,30.000000,44.000000,5.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
max,19999.000000,30.000000,1411.000000,12151.000000,9.000000,995.000000,425.000000,30.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000,1.000000,1.000000,1.0000,1.000000


In [467]:
## check for nulls
X[X.isnull().any(axis=1)]

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area,height,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other


In [468]:
## merge X, y on building_id
Xy=pd.merge(X,y,on='building_id')

## convert to int
Xy['count_families']=Xy.count_families.astype(np.int64)
Xy['has_secondary_use']=Xy.count_families.astype(np.int64)

In [469]:
## replace '0's as NANs
X.loc[X.geo_level_1_id == 0, 'geo_level_1_id'] = np.nan
X.loc[X.geo_level_2_id == 0, 'geo_level_2_id'] = np.nan
X.loc[X.geo_level_3_id == 0, 'geo_level_3_id'] = np.nan

In [470]:
## fill NaNs with median values
X.fillna(round(X.median()),inplace=True)

In [471]:
X.describe()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area,height,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.0000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.00000,10000.000000,10000.000000,10000.000000,10000.0000,10000.000000
mean,9987.160000,7.677400,300.462800,2683.293100,2.146700,25.393500,38.438100,4.653100,0.089700,0.762600,0.02990,0.019000,0.068800,0.072500,0.2561,0.087700,0.040000,0.013800,0.014100,0.984600,0.108600,0.067300,0.029400,0.006400,0.00070,0.00070,0.000800,0.000200,0.000200,0.0001,0.005300
std,5800.800829,5.829436,277.026219,2517.495486,0.736365,64.482893,21.265883,1.792842,0.285766,0.425511,0.17032,0.136532,0.253126,0.259327,0.4365,0.282872,0.195969,0.116666,0.117909,0.423297,0.311152,0.250553,0.168933,0.079748,0.02645,0.02645,0.028274,0.014141,0.014141,0.0100,0.072612
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,6.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
25%,4998.750000,3.000000,67.750000,615.000000,2.000000,10.000000,26.000000,4.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
50%,9963.500000,6.000000,225.000000,1948.000000,2.000000,15.000000,34.000000,5.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
75%,15044.750000,10.000000,466.000000,4158.000000,3.000000,30.000000,44.000000,5.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
max,19999.000000,30.000000,1411.000000,12151.000000,9.000000,995.000000,425.000000,30.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000,1.000000,1.000000,1.0000,1.000000


In [474]:
X.set_index(['building_id'],inplace=True)
X.head(5)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area,height,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
765,8.0,501.0,3237.0,2,25,33,4,d502,337f,7e76,b1b4,f962,3356,a779,0,1,0,0,0,0,0,0,0,0,0,c8e1,1.0,0.0,0,0,0,0,0,0,0,0,0,0
9470,7.0,288.0,1819.0,3,15,28,5,d502,337f,7e76,b1b4,f962,3356,a779,0,1,0,0,0,0,0,0,0,0,0,c8e1,0.0,0.0,0,0,0,0,0,0,0,0,0,0
14461,14.0,36.0,838.0,1,20,37,2,d502,337f,7e76,467b,441a,bfba,8e3f,0,1,0,0,1,0,0,0,0,0,0,c8e1,1.0,1.0,0,1,0,0,0,0,0,0,0,0
9317,8.0,81.0,805.0,2,5,18,3,d502,337f,7e76,b1b4,f962,3356,a779,0,1,0,0,0,0,0,0,0,0,0,c8e1,1.0,0.0,0,0,0,0,0,0,0,0,0,0
15941,23.0,380.0,4266.0,3,0,39,6,d502,337f,7e76,b1b4,f962,3356,a779,0,1,0,0,0,0,0,0,0,0,0,c8e1,1.0,0.0,0,0,0,0,0,0,0,0,0,0


In [476]:
y.set_index('building_id',inplace=True)

In [479]:
yT=np.ravel(y)

In [480]:
##test differenct category_encoders against the data

from sklearn.model_selection import train_test_split
import category_encoders as ce

from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

cols=['land_surface_condition','foundation_type','roof_type','ground_floor_type','other_floor_type','position','plan_configuration','legal_ownership_status']
encoder = ['BackwardDifferenceEncoder','BinaryEncoder','HashingEncoder', 'HelmertEncoder','OneHotEncoder','OrdinalEncoder','SumEncoder','PolynomialEncoder','BaseNEncodeLeaveOneOutEncoder']
train_accu=[]
test_accu=[]

encod = {"back":ce.BackwardDifferenceEncoder(cols),
       "bin":ce.BinaryEncoder(cols),
       "hash":ce.HashingEncoder(cols),
       "helm":ce.HelmertEncoder(cols),
       "1hot":ce.OneHotEncoder(cols),
       "ord":ce.OrdinalEncoder(cols),
       "sum":ce.SumEncoder(cols),
       "poly":ce.PolynomialEncoder(cols),
       "baseN":ce.BaseNEncoder(cols)}

structs=['back','bin','hash','helm','1hot','ord','sum','poly','baseN']
        
        
for elem in structs:
    #method = encod[elem]
    method = 'ce.%s'%encod[elem]
    method=eval(method)

    XT = method.fit_transform(X)

    X_train, X_test, y_train, y_test=train_test_split(XT, yT, test_size=0.33, random_state=7)
        
    lr = linear_model.LogisticRegression()
    lr.fit(X_train, y_train) 
    train_accu.append(metrics.accuracy_score(y_train,lr.predict(X_train)))
    test_accu.append(metrics.accuracy_score(y_test, lr.predict(X_test)))  

##print metrics    
print('**Encoder**','**Train Accuracy**', '**Test Accuracy**')
for i in range(9):
    print(encoder[i],train_accu[i],test_accu[i])
    

**Encoder** **Train Accuracy** **Test Accuracy**
BackwardDifferenceEncoder 0.5944776119402985 0.5960606060606061
BinaryEncoder 0.5941791044776119 0.6006060606060606
HashingEncoder 0.5855223880597015 0.5924242424242424
HelmertEncoder 0.5988059701492537 0.5984848484848485
OneHotEncoder 0.5953731343283583 0.6015151515151516
OrdinalEncoder 0.5867164179104477 0.5990909090909091
SumEncoder 0.5929850746268657 0.593939393939394
PolynomialEncoder 0.5908955223880596 0.6003030303030303
BaseNEncodeLeaveOneOutEncoder 0.5941791044776119 0.6006060606060606
